In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
train_df = pd.read_csv("../input/eurecom-aml-2022-challenge-1/public/train.csv", low_memory=True).iloc[:,1:]  # drop index column

In [ ]:
train_df.head()

# Exploratory Data Analysis

Let's inspect geographical data
First, we look at temperature in different areas of the world


In [ ]:
fig, ax = plt.subplots()
train_df.plot(kind="scatter", x="fact_longitude", y="fact_latitude", alpha=0.4, c="fact_temperature",
              cmap=plt.get_cmap("jet"), colorbar=True, figsize=(15,7), ax=ax)
plt.tight_layout()
plt.grid()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel("Longitude", fontsize=20)
plt.ylabel("Latitude", fontsize=20)
plt.show()

Now, look at climate pressure in different areas


In [ ]:
fig, ax = plt.subplots()
train_df.plot(kind="scatter", x="fact_longitude", y="fact_latitude", alpha=0.4, c="climate_pressure",
              cmap=plt.get_cmap("jet"), colorbar=True, figsize=(15,7), ax=ax)
plt.tight_layout()
plt.grid()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel("Longitude", fontsize=20)
plt.ylabel("Latitude", fontsize=20)
plt.show()

We can see how training data is coming from all around the world, with some exceptions: center and north South America and center of Africa.

## Target Variable

In [ ]:
import seaborn as sns

def plot_violin_plot(column):
  sns.set_theme(style="whitegrid")
  ax = sns.violinplot(y=column)

plt.figure(figsize=(8,6))
plot_violin_plot(train_df["fact_temperature"])
plt.show()

In [ ]:
percentile_99_99 = train_df["fact_temperature"].quantile(0.9999)
percentile_0_01 = train_df["fact_temperature"].quantile(0.0001)
percentile_99_9 = train_df["fact_temperature"].quantile(0.999)
percentile_0_1 = train_df["fact_temperature"].quantile(0.001)
percentile_99 = train_df["fact_temperature"].quantile(0.99)
percentile_1 = train_df["fact_temperature"].quantile(0.1)


print(f"Percentile 99.99 %: {percentile_99_99}")
print(f"Percentile 0.01 %: {percentile_0_01}")
print(f"Percentile 99.9 %: {percentile_99_9}")
print(f"Percentile 0.1 %: {percentile_0_1}")
print(f"Percentile 99 %: {percentile_99}")
print(f"Percentile 1 %: {percentile_1}")

print(f"Percentile 99.99 % data over: { ( train_df['fact_temperature'] >= percentile_99_99).sum()} ")
print(f"Percentile 0.01 % data under: { ( train_df['fact_temperature'] <= percentile_0_01).sum()} ")
print(f"Percentile 99.9 % data over: { ( train_df['fact_temperature'] >= percentile_99_9).sum()} ")
print(f"Percentile 0.1 % data under: { ( train_df['fact_temperature'] <= percentile_0_1).sum()} ")
print(f"Percentile 99 % data over: { ( train_df['fact_temperature'] >= percentile_99).sum()} ")
print(f"Percentile 1 % data under: { ( train_df['fact_temperature'] <= percentile_1).sum()}")

# Data cleaning

We check that all features are numerical -> yes, so we don't need encoding

In [ ]:
train_df.info()

In [ ]:
print(f"NaN values before dealing with them: {train_df.isna().sum().sum()}")

All NaN values are in the following records


In [ ]:
print(f"gfs records not available: {(train_df['gfs_available'] == 0).sum()}")
print(f"cmc records not available: {(train_df['cmc_available'] == 0).sum()}")
print(f"wrf records not available: {(train_df['wrf_available'] == 0).sum()}")

We create a categorical variable called 'flag', which is equal to 0 if the record does not have any null features, 1 if cmc features are missing, 2 if wrf features are missing and 3 if both cmc and wrf features are missing. 

This variable is created only for visualization purposes.


In [ ]:
nan_points_cmc = (train_df['cmc_available'] == 0).replace({True: 1, False: 0})
nan_points_wrf = (train_df['wrf_available'] == 0).replace({True: 2, False: 0})
nan_points_flag = pd.concat([nan_points_cmc, nan_points_wrf]).groupby(level=0).sum()
train_df["flag"] = nan_points_flag


In [ ]:
train_df.plot(kind="scatter", x="fact_longitude", y="fact_latitude", alpha=0.4, c="flag",
              cmap=plt.get_cmap("jet"), colorbar=True, figsize=(15,7))
plt.show()

Blue points represent regions with no null records. We can see that half of Australia points are records with NaN values (yellow points). 

In [ ]:
# drop available data feature and its flag which was useful for visualization only
train_df = train_df.drop(columns=["gfs_available", "cmc_available", "wrf_available", "flag"])

Dealing with NaN values by deleting them will lead us to a loss of information about half of Oceania. Thus, we will sort by latitude and longitude, then use bfill and ffill approaches, then restore original order by index. With bfill and ffill we mean backward and forward fill. The former uses next valid observation to fill the gap, while the latter propagates last valid observation forward

In [ ]:
train_df = train_df.sort_values(["fact_latitude", "fact_longitude"])
train_df = train_df.fillna(method="bfill")
train_df = train_df.fillna(method="ffill")
train_df = train_df.sort_index()
train_df

As a result, we won't have any NaN record. 

In [ ]:
train_df.isna().sum().sum()

We remove the duplicates from our training set

In [ ]:
train_df.drop_duplicates(inplace=True, ignore_index=True)

# Outliers removal

IQR method was tried, but we chose not to include it since it removes very low and very high temperatures from the training, which is useful information

In [ ]:
'''
percentile25 = train_df['fact_temperature'].quantile(0.25)
percentile75 = train_df['fact_temperature'].quantile(0.75)
iqr = percentile75 - percentile25

upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

print(f"Temperature upper limit: {upper_limit}")
print(f"Temperature lower limit: {lower_limit}")

print(f"Number of data that exceeds temperature upper limit: {train_df[train_df['fact_temperature'] > upper_limit].shape[0]}")
print(f"Number of data that exceeds temperature lower limit: {train_df[train_df['fact_temperature'] < lower_limit].shape[0]}")

print(f"Len of train_df: {train_df.shape[0]} ")

train_df =  train_df.drop( train_df[train_df['fact_temperature'] > upper_limit].index) # drop record over the upper limit
train_df =  train_df.drop( train_df[train_df['fact_temperature'] < lower_limit].index) # drop record under the upper limit

print(f"Len of train_df: {train_df.shape[0]}")
'''



# Features Engineering

We create new features from the timestamp feature. In particular, we extract the month and the our from the UNIX timestamp, since we believe they are the most useful information to predict the temperature.
Then, we one hot encoded them not to create a misleading relation between them and the target variable. For example an higher month does not translate to higher/lower temperatures.

In [ ]:
from datetime import datetime

def get_hour(date, n_intervals = 6):
    
    hour = date.hour
    hours_in_interval = 24 // n_intervals

    return hour // hours_in_interval


def get_categorical_month_and_hour(df):

    datetimes = df["fact_time"].apply(datetime.utcfromtimestamp)

    df["month"] = datetimes.apply(lambda date: date.month)
    df["hour"] = datetimes.apply(get_hour) # get hour in intervals

    return pd.get_dummies(data=df, columns=['month', 'hour'])

In [ ]:
train_df = get_categorical_month_and_hour(train_df)

train_df = train_df.drop(columns=["fact_time"])

# Feature selection

Plotting the Covariance matrix : 

In [ ]:
plt.figure(figsize = (13,8))
corr_matrix = train_df.corr().abs()
sns.heatmap(corr_matrix)
plt.tight_layout()
plt.show()

Plotting the covariance matrix between some gfs temperature features to vizualize the covariance between them.

In [ ]:
columns_gfs_corr = ['gfs_temperature_7000','gfs_temperature_20000','gfs_temperature_35000','gfs_temperature_50000','gfs_temperature_65000','gfs_temperature_80000','gfs_temperature_92500']
corr_gfs_temp = train_df[columns_gfs_corr].corr()
sns.heatmap(corr_gfs_temp)
plt.xticks()
plt.tight_layout()
plt.show()

We check the linear correlation between each pair of features. If two features are highly correlated, then we drop one of them.

In [ ]:
def get_correlated_features(df, threshold, method='pearson'):
    corr_matrix = df.corr(method=method).abs()
    high_corr_var=np.where(corr_matrix>threshold)
    high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
    # features and how many times they appear in tuples of correlated features
    features_count = pd.Series(sum(np.array(high_corr_var).tolist(), [])).value_counts()
    to_del = set()
    for pair in high_corr_var:
        if features_count[pair[0]] > features_count[pair[1]]:
            to_del.add(pair[0])
        else:
            to_del.add(pair[1])
    return list(to_del)

In [ ]:
df_cmc_gfs_wrf = train_df[train_df.columns.values[train_df.columns.str.contains("gfs") | 
                                                  train_df.columns.str.contains("cmc") |
                                                  train_df.columns.str.contains("wrf")]]

# pearson: linear relation between two variables
features_to_remove_pearson = get_correlated_features(df_cmc_gfs_wrf, threshold=0.8, method='pearson')

features_to_remove = features_to_remove_pearson
features_to_remove

We check correlation between target variable and other features


In [ ]:
corr_target = train_df.corrwith(train_df["fact_temperature"]).sort_values(ascending=False)
corr_target

We finally drop the features highly correlated with other features

In [ ]:
final_train_df = train_df.drop(columns=features_to_remove)
final_train_df

# Scaling of features

We used Standard Scaler in order to use a common scale for the features, without distorting differences in the range of values.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Get X_train and y_train and normalize
X_train = final_train_df.drop(columns=['fact_temperature'])
y_train = final_train_df['fact_temperature']

sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)

scy = StandardScaler()
scy.fit(y_train.to_numpy().reshape(-1, 1))
y_train = scy.transform(y_train.to_numpy().reshape(-1, 1))

# PCA feature selection

In [ ]:
'''from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


X_sf = X_train.copy()


percentage_of_variance = 0.95
pca = PCA(percentage_of_variance)
X_train_pca = pca.fit_transform(X_sf)
'''

# Forward selection

In [ ]:
'''
# Feature selection iterative model
import statsmodels.api as sm
from statsmodels.formula.api import ols

# print(X_train)
# print(y_train)

remaining_features = list(X_train.columns)

data = X_train.copy()
data["y"] = y_train[:]

print(remaining_features)

alpha = 0.025

# provo tutte le colonne e se il p value è minore di 0.05 aggiungo a una lista, poi prendo la colonna con il p-value minore
# e la aggiungo al mio modello. Poi riparto con il mio modello aggiornato

selected_features_forward = []
while remaining_features: 
  PF = []  #list of (P value, feature)
  print(f"SELECTED FEATURES: {selected_features_forward}")
  for f in remaining_features:
    temp = selected_features_forward + [f]  #temporary list of features+

    log_reg = sm.Logit(y_train, X_train[temp]).fit()

    p_values = log_reg.pvalues
    min_p_value = np.min(log_reg.pvalues)

    if min_p_value < alpha:
       PF.append((min_p_value,f))
  if PF:  #if not empty
     PF.sort(reverse=True)
     (best_pval, best_f) = PF.pop()
     remaining_features.remove(best_f)
     print('selected feature {} with p-value = {:.2E}'.
            format(best_f, best_pval))
     selected_features_forward.append(best_f)
  else:
     break

print("---------------------------------------------------")
print(selected_features_forward)
print(data)

'''

# Training

In [ ]:
# grid search cv

param_grid = {
            "alpha": [0.1, 0.01, 0.001]
            }
gs = GridSearchCV(estimator=Lasso(), param_grid=param_grid, 
                  scoring="neg_root_mean_squared_error",cv=5, verbose=True)
gs.fit(X_train,y_train)

In [ ]:
print(f"Best set of parameters: {gs.best_params_}")
print(f"Best score: {gs.best_score_}")

In [ ]:
# Now train with full train dataset and best parameters
model = Lasso(alpha=0.001)
model.fit(X_train, y_train)

In [ ]:
# Train using a neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping


def model():
    model = Sequential()
    model.add(Dense(64, activation="relu", kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dense(32, activation="relu",  kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dense(32, activation="relu",  kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dense(16, activation="relu",  kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dense(1))
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

model = model()

callback = EarlyStopping(monitor='loss', patience=3)

history = model.fit(
        X_train,
        y_train,
        epochs=50,
        batch_size=1024,
        validation_split=0.2,
        verbose=True,
        callbacks=[callback]
)

In [ ]:
def plot_history(losses, val_losses):
    plt.figure(figsize=(12,8))
    plt.plot(losses)
    plt.plot(val_losses)
    plt.legend(["train_loss", "val_loss"], fontsize=20)
    plt.xlabel("Epochs", fontsize=20)
    plt.ylabel("Loss", fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.grid()
    plt.savefig('train_val_loss.png')
    plt.show()
    
plot_history(
        history.history["loss"],
        history.history["val_loss"]
    )



# Test

We load the test model and plot the distribution of the measurements around the globe

In [ ]:
test_df = pd.read_csv("../input/eurecom-aml-2022-challenge-1/public/test_feat.csv", low_memory=True)
test_df.plot(kind="scatter", x="fact_longitude", y="fact_latitude", alpha=0.4,
              cmap=plt.get_cmap("jet"), figsize=(15,7))
plt.show()
indices = test_df.pop("index")

Preprocessing of test data

In [ ]:
# add info about month and hour
test_df = get_categorical_month_and_hour(test_df)
# drop features
final_test_df = test_df.drop(columns=features_to_remove)
final_test_df = final_test_df.drop(columns=["fact_time", "gfs_available", "cmc_available", "wrf_available"])

# Get X_test and normalize
X_test = final_test_df.to_numpy()
X_test = sc.transform(X_test)

Prediction of test data

In [ ]:
y_pred = model.predict(X_test)
y_pred = scy.inverse_transform(y_pred)

In [ ]:
submission_df = pd.DataFrame(data={'index': indices.values,
                                   'fact_temperature': y_pred.squeeze()})

# Save the predictions into a csv file
# Notice that this file should be saved under the directory `/kaggle/working` 
# so that you can download it later
submission_df.to_csv("/kaggle/working/submission.csv", index=False)